In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import geopandas as gpd 
import numpy as np

import matplotlib.pyplot as plt
import plotly.express as px

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import warnings
warnings.filterwarnings("ignore")

In [3]:
import sys
sys.path.append("../") 

from utils.paths import make_dir_line

modality = 'u'
project = 'Ciencia de los datos'
data = make_dir_line(modality, project)

raw = data('raw')
processed = data('processed')
models = data('models')

In [4]:
geo = gpd.read_file(raw / "Limiti01012023_g/RipGeo01012023_g/RipGeo01012023_g_WGS84.shp")
geo.geometry = geo.geometry.to_crs(epsg = 4326)
geo = geo.loc[:,['COD_RIP','DEN_RIP']]

lista_verificacion = geo['DEN_RIP'].unique()
print('Lista de verificacion: ', lista_verificacion)

geo.head()

Lista de verificacion:  ['Nord-Ovest' 'Nord-Est' 'Centro' 'Sud' 'Isole']


,COD_RIP,DEN_RIP
0,1,Nord-Ovest
1,2,Nord-Est
2,3,Centro
3,4,Sud
4,5,Isole


In [5]:
ter = pd.read_parquet(processed / 'ter2.parquet.gzip')
ter.rename(columns={'Territory':'DEN_RIP'}, inplace=True)
ter = ter[ter['DEN_RIP'].isin(lista_verificacion)]
ter.head()

,ITTER107,DEN_RIP
4,ITC,Nord-Ovest
62,ITD,Nord-Est
120,ITE,Centro
174,ITF,Sud
234,ITG,Isole


In [6]:
ter.shape, geo.shape

((5, 2), (5, 2))

In [7]:
df_aux = pd.merge(ter, geo, on=['DEN_RIP'])
df_aux

,ITTER107,DEN_RIP,COD_RIP
0,ITC,Nord-Ovest,1
1,ITD,Nord-Est,2
2,ITE,Centro,3
3,ITF,Sud,4
4,ITG,Isole,5


In [8]:
geo = gpd.read_file(raw / "Limiti01012023_g/Reg01012023_g/Reg01012023_g_WGS84.shp")
geo.geometry = geo.geometry.to_crs(epsg = 4326)

lista_verificacion = geo['DEN_REG'].unique()
print('Lista de verificacion: ', lista_verificacion)

geo.head()

Lista de verificacion:  ['Piemonte' "Valle d'Aosta" 'Lombardia' 'Trentino-Alto Adige' 'Veneto'
 'Friuli Venezia Giulia' 'Liguria' 'Emilia-Romagna' 'Toscana' 'Umbria'
 'Marche' 'Lazio' 'Abruzzo' 'Molise' 'Campania' 'Puglia' 'Basilicata'
 'Calabria' 'Sicilia' 'Sardegna']


,COD_RIP,COD_REG,DEN_REG,Shape_Leng,Shape_Area,geometry
0,1,1,Piemonte,1.236800e+06,2.539388e+10,"POLYGON ((8.44976 46.46176, 8.46176 46.45081, ..."
1,1,2,Valle d'Aosta,3.109681e+05,3.258838e+09,"POLYGON ((7.58857 45.97075, 7.58981 45.97073, ..."
2,1,3,Lombardia,1.410223e+06,2.386232e+10,"MULTIPOLYGON (((8.81642 45.02231, 8.81427 45.0..."
3,2,4,Trentino-Alto Adige,8.008937e+05,1.360755e+10,"POLYGON ((12.20511 47.08653, 12.20668 47.08627..."
4,2,5,Veneto,1.054587e+06,1.834355e+10,"POLYGON ((12.50591 46.67839, 12.50603 46.67803..."


In [9]:
ter = pd.read_parquet(processed / 'ter2.parquet.gzip')
ter = ter.dropna()
ter.rename(columns={'Territory':'DEN_REG'}, inplace=True)

patron = r'^IT[C,D,E,F,G][0-9]$'  # Patrón para buscar "IT" seguido de una letra entre D y G y un solo dígito
ter = ter[ter["ITTER107"].str.match(patron)]

ter.head()

,ITTER107,DEN_REG
6,ITC1,Piemonte
24,ITC2,Valle D'Aosta / Vallée D'Aoste
28,ITC3,Liguria
38,ITC4,Lombardia
66,ITD1,Provincia Autonoma Bolzano / Bozen


In [10]:
ter

,ITTER107,DEN_REG
6,ITC1,Piemonte
24,ITC2,Valle D'Aosta / Vallée D'Aoste
28,ITC3,Liguria
38,ITC4,Lombardia
66,ITD1,Provincia Autonoma Bolzano / Bozen
70,ITD2,Trentino-Alto Adige
74,ITD3,Veneto
90,ITD4,Friuli Venezia Giulia
100,ITD5,Emilia-Romagna
122,ITE1,Toscana


In [11]:
ter.shape, geo.shape

((21, 2), (20, 6))

In [12]:
df_aux2 = pd.merge(ter, geo, on=['DEN_REG'])
df_aux2 = df_aux2.sort_values(by=['COD_REG'], ascending=True)
df_aux2

,ITTER107,DEN_REG,COD_RIP,COD_REG,Shape_Leng,Shape_Area,geometry
0,ITC1,Piemonte,1,1,1.236800e+06,2.539388e+10,"POLYGON ((8.44976 46.46176, 8.46176 46.45081, ..."
2,ITC4,Lombardia,1,3,1.410223e+06,2.386232e+10,"MULTIPOLYGON (((8.81642 45.02231, 8.81427 45.0..."
3,ITD2,Trentino-Alto Adige,2,4,8.008937e+05,1.360755e+10,"POLYGON ((12.20511 47.08653, 12.20668 47.08627..."
4,ITD3,Veneto,2,5,1.054587e+06,1.834355e+10,"POLYGON ((12.50591 46.67839, 12.50603 46.67803..."
5,ITD4,Friuli Venezia Giulia,2,6,6.708207e+05,7.934116e+09,"MULTIPOLYGON (((13.77538 45.61068, 13.77538 45..."
1,ITC3,Liguria,1,7,8.196598e+05,5.414542e+09,"MULTIPOLYGON (((9.85132 44.02340, 9.85122 44.0..."
6,ITD5,Emilia-Romagna,2,8,1.180343e+06,2.249987e+10,"MULTIPOLYGON (((10.48080 44.18949, 10.48069 44..."
7,ITE1,Toscana,3,9,1.306243e+06,2.298404e+10,"MULTIPOLYGON (((11.11471 42.25911, 11.11625 42..."
8,ITE2,Umbria,3,10,6.197684e+05,8.464380e+09,"MULTIPOLYGON (((12.43119 43.59136, 12.43030 43..."
9,ITE3,Marche,3,11,6.170373e+05,9.343412e+09,"POLYGON ((12.76834 43.96594, 12.76911 43.96585..."


In [13]:
df_aux2.shape

(19, 7)

In [14]:
geo = gpd.read_file(raw / "Limiti01012023_g/ProvCM01012023_g/ProvCM01012023_g_WGS84.shp")
geo.geometry = geo.geometry.to_crs(epsg = 4326)
geo['DEN_PROV'] = np.where(geo['DEN_PROV'] == '-', np.nan, geo['DEN_PROV'])
geo = geo.dropna(subset=['DEN_PROV'])

lista_verificacion = geo['DEN_UTS'].unique()
print('Lista de verificacion: ', lista_verificacion)

geo.head()

Lista de verificacion:  ['Vercelli' 'Novara' 'Cuneo' 'Asti' 'Alessandria' 'Aosta' 'Imperia'
 'Savona' 'La Spezia' 'Varese' 'Como' 'Sondrio' 'Bergamo' 'Brescia'
 'Pavia' 'Cremona' 'Mantova' 'Bolzano' 'Trento' 'Verona' 'Vicenza'
 'Belluno' 'Treviso' 'Padova' 'Rovigo' 'Udine' 'Gorizia' 'Trieste'
 'Piacenza' 'Parma' "Reggio nell'Emilia" 'Modena' 'Ferrara' 'Ravenna'
 "Forli'-Cesena" 'Pesaro e Urbino' 'Ancona' 'Macerata' 'Ascoli Piceno'
 'Massa Carrara' 'Lucca' 'Pistoia' 'Livorno' 'Pisa' 'Arezzo' 'Siena'
 'Grosseto' 'Perugia' 'Terni' 'Viterbo' 'Rieti' 'Latina' 'Frosinone'
 'Caserta' 'Benevento' 'Avellino' 'Salerno' "L'Aquila" 'Teramo' 'Pescara'
 'Chieti' 'Campobasso' 'Foggia' 'Taranto' 'Brindisi' 'Lecce' 'Potenza'
 'Matera' 'Cosenza' 'Catanzaro' 'Trapani' 'Agrigento' 'Caltanissetta'
 'Enna' 'Ragusa' 'Siracusa' 'Sassari' 'Nuoro' 'Pordenone' 'Isernia'
 'Oristano' 'Biella' 'Lecco' 'Lodi' 'Rimini' 'Prato' 'Crotone'
 'Vibo Valentia' 'Verbano-Cusio-Ossola' 'Monza e della Brianza' 'Fermo'
 'Barlett

,COD_RIP,COD_REG,COD_PROV,COD_CM,COD_UTS,DEN_PROV,DEN_CM,DEN_UTS,SIGLA,TIPO_UTS,Shape_Area,geometry
1,1,1,2,0,2,Vercelli,-,Vercelli,VC,Provincia,2.082097e+09,"POLYGON ((8.20447 45.93567, 8.21365 45.92490, ..."
2,1,1,3,0,3,Novara,-,Novara,NO,Provincia,1.341318e+09,"POLYGON ((8.49688 45.83934, 8.49996 45.83402, ..."
3,1,1,4,0,4,Cuneo,-,Cuneo,CN,Provincia,6.898501e+09,"MULTIPOLYGON (((7.46667 44.77289, 7.46997 44.7..."
4,1,1,5,0,5,Asti,-,Asti,AT,Provincia,1.508923e+09,"POLYGON ((8.04681 45.12815, 8.04572 45.12300, ..."
5,1,1,6,0,6,Alessandria,-,Alessandria,AL,Provincia,3.560310e+09,"POLYGON ((8.40549 45.20148, 8.41749 45.19846, ..."


In [30]:
ter = pd.read_parquet(processed / 'ter2.parquet.gzip')
ter = ter.dropna()
ter.rename(columns={'Territory':'DEN_PROV'}, inplace=True)

patron = r'^IT[C,D,E,F,G]((\d[A-Z])|(\d{2}))$'
ter = ter[ter["ITTER107"].str.match(patron)]

ter.head()

,ITTER107,DEN_PROV
8,ITC11,Torino
10,ITC12,Vercelli
12,ITC13,Biella
14,ITC14,Verbano-Cusio-Ossola
16,ITC15,Novara


In [31]:
ter.shape, geo.shape

((103, 2), (93, 12))

In [32]:
df_aux3 = pd.merge(ter, geo, on=['DEN_PROV'])
df_aux3 = df_aux3.sort_values(by=['COD_PROV'], ascending=True)
filas_seleccionadas = df_aux3.iloc[1:10] 
print(filas_seleccionadas)

   ITTER107     DEN_PROV  COD_RIP  COD_REG  COD_PROV  COD_CM  COD_UTS DEN_CM   
3     ITC15       Novara        1        1         3       0        3      -  \
4     ITC16        Cuneo        1        1         4       0        4      -   
5     ITC17         Asti        1        1         5       0        5      -   
6     ITC18  Alessandria        1        1         6       0        6      -   
7     ITC31      Imperia        1        7         8       0        8      -   
8     ITC32       Savona        1        7         9       0        9      -   
9     ITC34    La Spezia        1        7        11       0       11      -   
10    ITC41       Varese        1        3        12       0       12      -   
11    ITC42         Como        1        3        13       0       13      -   

        DEN_UTS SIGLA   TIPO_UTS    Shape_Area   
3        Novara    NO  Provincia  1.341318e+09  \
4         Cuneo    CN  Provincia  6.898501e+09   
5          Asti    AT  Provincia  1.508923e+09   

In [33]:
df_aux3.shape

(83, 13)

In [34]:
geo = geo.sort_values(by=['COD_PROV'], ascending=True)
filas_seleccionadas = geo.iloc[1:10] 
print(filas_seleccionadas)

    COD_RIP  COD_REG  COD_PROV  COD_CM  COD_UTS     DEN_PROV DEN_CM   
2         1        1         3       0        3       Novara      -  \
3         1        1         4       0        4        Cuneo      -   
4         1        1         5       0        5         Asti      -   
5         1        1         6       0        6  Alessandria      -   
6         1        2         7       0        7        Aosta      -   
7         1        7         8       0        8      Imperia      -   
8         1        7         9       0        9       Savona      -   
10        1        7        11       0       11    La Spezia      -   
11        1        3        12       0       12       Varese      -   

        DEN_UTS SIGLA   TIPO_UTS    Shape_Area   
2        Novara    NO  Provincia  1.341318e+09  \
3         Cuneo    CN  Provincia  6.898501e+09   
4          Asti    AT  Provincia  1.508923e+09   
5   Alessandria    AL  Provincia  3.560310e+09   
6         Aosta    AO  Provincia  3.258